# Introduction to PyBullet (Optional)

[PyBullet](https://pybullet.org/wordpress/) is a "rigid body dynamics" simulator, it means that it can simulate any articulated, rigid, robot. For example our NYU finger but also more complex robots, such as a humanoid, a quadruped or a hand. It can also simulate other objects and several robots at the same time.

## How does it work?

In a nutshell, every simulator functions as follows:
* First a description of the objects and robots to simulate are given, including their dynamic properties (masses, center of mass), their "boundaries" (to detect collisions between objects) and graphic files to visualize the robot. The visualization is often different than what is simulated (i.e. we often simplify the geometry of the robot, e.g. using cylinders, for the simulation but display all the details in the visualization)

* At each instant of time, the simulator keeps track of all the collisions between all the objects, to know where to compute contact forces. Typically the collisions are simplified to make the simulation fast enough.

* The simulator then computes all the forces exerted on all the objects and robots (e.g. gravity, contact between objects, etc) and uses Netwon's law of motion to compute the next position and velocity of all the objects.

The user can then decide to apply forces/torques on certain joints, like a motor would do and read all the available information from the simulator to emulate real sensors.

As a user, we then need to write a "controller" that decides what to do at each simulation step.


In [ ]:
# let's first import pybullet in the notebook
import pybullet as p

#setup nice plotting
jup_type = !jupyter --version
if 'jupyterlab' in ' '.join(jup_type):
    %matplotlib widget
else:
    %matplotlib notebook

# we also import other useful libraries
import time
import numpy as np
import matplotlib as mp
import matplotlib.pyplot as plt

In [ ]:
# now we can start pybullet and set a few parameters
# an empty simulation window should appear

# Connect to pybullet and setup simulation parameters.
p.connect(p.GUI)

# we set gravity
p.setGravity(0, 0, -9.81)
# we set the integration step to 1ms (each time we step the simulation it will advance by 1ms)
p.setPhysicsEngineParameter(fixedTimeStep=1.0/1000.0, numSubSteps=1)
# Disable the gui controller as we don't use them.
p.configureDebugVisualizer(p.COV_ENABLE_GUI,0)

We can now import our robot, using a [URDF file](http://wiki.ros.org/urdf/XML/model) (Unified Robot Description Format), a commonly used format to describe articulated robots.

It describes the links of the robot and the joints connecting the links. It allows to specify the dynamic parameters (mass, center of mass and inertia), the collision model and the visualization.

You can take a look at the URDF file of the [NYU finger](./urdf/fingeredu.urdf). We will see in our next lectures how we can describe any robot like that.

In [ ]:
# let's load our robot

# Zoom onto the robot.
p.resetDebugVisualizerCamera(1.0, 50, -35, (0., 0., 0.))

# we set the initial position and orientation of the robot
robotStartPosition = [0.,0,.0]
robotStartOrientation = p.getQuaternionFromEuler([0,0,0])

# we load the robot - the robot should be attached to the ground
# so we set useFixedBase to True
robotId = p.loadURDF('./urdf/fingeredu.urdf', robotStartPosition,
                robotStartOrientation, flags=p.URDF_USE_INERTIA_FROM_FILE,
                useFixedBase=True)

# you should now see our NYU finger in the pybullet window
# you should however not be able to do much because the simulation is not running yet

In [ ]:
# we can now query some information about the robot
nj = p.getNumJoints(robotId)
print('the robot has: ' + str(nj) + ' joints\n')
print('the joint names are:')
for i in range(nj):
    print(p.getJointInfo(robotId, i)[1].decode('UTF-8'))

Here we see that the number of joints of the robot known to PyBullet is not the number of actuated joints in the robot. 

The first joint "base_to_finger" corresponds to a "fixed joint" that describe the attachement between the base of the robot and the first join.

The last joint "finger_lower_to_tip_joint" corresponds to a "fixed joint" that allows to attach the fingertip to the leg.

These two joints are "fake". They are useful for the simulation and to describe the robot, find coordinate frames (we will discuss this in the next lectures!)

The joints we care about are:
* finger_base_to_upper_joint which is the "shoulder"
* finger_upper_to_middle_joint which is the "hip"
* finger_middle_to_lower_joint which is the "elbow"

In [ ]:
# we are going to store the ids of the joints we care about
joint_names = [
            'finger_base_to_upper_joint',
            'finger_upper_to_middle_joint',
            'finger_middle_to_lower_joint',
        ]

# a map from names to ids
bullet_joint_map = {}
for ji in range(p.getNumJoints(robotId)):
    bullet_joint_map[p.getJointInfo(robotId, ji)[1].decode('UTF-8')] = ji

# a list of ids we are interested in
bullet_joint_ids = np.array([bullet_joint_map[name] for name in joint_names])
num_joints = bullet_joint_ids.size

In [ ]:
# we can now run the simulation of the robot for 10 seconds

# first we need to disable the velocity control on the joints - we set 0 torques on each joint
p.setJointMotorControlArray(robotId, bullet_joint_ids, p.VELOCITY_CONTROL, forces=np.zeros(num_joints))

run_time = 10.
dt = 0.001
num_steps = int(run_time/dt)

### this is the simulation loop ###
### we do one step of simulation num_steps times ###
for i in range(num_steps):
    time.sleep(dt)
    p.stepSimulation()

In [ ]:
# we can now reset the robot to a different position and simulate again
# now we will also read the joint position and velocities directly from the simulator
new_desired_position = [0.0,1.,1.]
for i in range(num_joints):
    p.resetJointState(robotId, i+1, new_desired_position[i], 0.)

measured_positions = np.zeros([num_steps,3])
measured_velocities = np.zeros_like(measured_positions)


### this is the simulation loop ###
### we do one step of simulation num_steps times ###
for i in range(num_steps):
    time.sleep(dt)
    p.stepSimulation()
    
    # we access the joints and read each joint position and velocity #
    joint_states = p.getJointStates(robotId, bullet_joint_ids)
    for j in range(num_joints):
        measured_positions[i,j] = joint_states[j][0]
        measured_velocities[i,j] = joint_states[j][1]

We can now plot the position of each joint as a function of time

In [ ]:
# we create the time vector (as we simulated for 10 seconds)
time = np.linspace(0., 10., num=num_steps)

# we create a figure using matplotlib
plt.figure(figsize=[6, 12])

# for each joint we plot the measured position as a function of time
for i in range(3):
    plt.subplot(3,1,i+1)
    plt.plot(time, measured_positions[:,i])
    plt.ylabel(joint_names[i] + ' [rad]')

# adding some labels
plt.xlabel('Time[s]')
plt.title('joint positions')

We can now plot the velocities of each joint as a function of time

In [ ]:
# we do the same for velocities
plt.figure(figsize=[6, 12])
for i in range(3):
    plt.subplot(3,1,i+1)
    plt.plot(time, measured_velocities[:,i])
    plt.ylabel(joint_names[i] + ' [rad/s]')
plt.xlabel('Time[s]')
plt.title('joint velocities')

PyBullet is a very complete simulator and we only introduce some very basic functionalities. For more information about the API of the simulator please check [this page](https://docs.google.com/document/d/10sXEhzFRSnvFcl3XxNGhnD4N2SedqwdAvK3dsihxVUA/edit#heading=h.2ye70wns7io3)

More information about the simulator, robot models, etc can be found on the [simulator website](https://pybullet.org/wordpress/)